# Lab 6 

Ali Tejani, amt3639

Caroline Yao, chy253

Allen Hwang, ah45755

## Problem 1

### Estimating beta with linear regression

In [83]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge

In [84]:
n = 51
p = 50

In [154]:
def estimate_b(reg,n,p,seed=None):
    np.random.seed(seed=seed)
    b_hat = []
    for _ in range(100):
        X = np.random.normal(0,1,(n,p))
        e = np.random.normal(0,.25,(n,1))
        b = np.ones((p,1))
        y = np.dot(X,b) + e
        reg.fit(X,y)
        reg.coef_
        b_hat.append(reg.coef_)
    b_hat = np.array(b_hat)
    print b_hat.mean(axis=0).mean()
    print b_hat.var(axis=0).mean()

In [165]:
estimate_b(LinearRegression(fit_intercept=False),n,p,seed=1)

1.00655839826
0.227150750588


In [166]:
alphas = [0.01,0.1,1,10,100]
for a in alphas:
    estimate_b(Ridge(alpha=a),n,p,seed=1)

0.982576226953
0.0641155927754
0.950031875305
0.0392228415033
0.863646895296
0.0623827708922
0.642854600564
0.091449034224
0.268810522308
0.0409478695197


## Problem 2: Chapter 6, Problem 9

### a.  Split the data set into a training set and a test set

In [219]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [309]:
college = pd.read_csv('College.csv',index_col=0)
college['Private'] = college['Private'].replace('Yes',1).replace('No',0)
collegeX = college[[i for i in college.axes[1] if i != 'Apps']]
collegeY = college['Apps']
college_trainX,college_testX,college_trainY,college_testY = train_test_split(collegeX,collegeY)

### b. Fit a linear model using least squares on the training set, and report the test error obtained

In [310]:
def reg_score(reg):
    reg.fit(college_trainX,college_trainY)
    print reg.score(college_testX,college_testY)

In [311]:
reg_score(LinearRegression())

0.946463693802


### c.  Fit a ridge regression model on the training set, with λ chosen by cross-validation. Report the test error obtained.

In [312]:
from sklearn.linear_model import RidgeCV,LassoCV
ridge_reg = RidgeCV(alphas=[0.001,0.005,0.01,0.05,0.1,0.5,1,10,100])
reg_score(ridge_reg)
ridge_reg.alpha_

0.946548958417


10.0

### d.  Fit a lasso model on the training set, with λ chosen by crossvalidation. Report the test error obtained, along with the number of non-zero coefficient estimates.

In [319]:
lasso_reg = LassoCV(alphas=[0.0001,0.001,0.005,0.01,0.05,0.1,0.5,1,10,100])
reg_score(lasso_reg)
print lasso_reg.alpha_
print lasso_reg.coef_

0.94649594954
1.0
[ -5.12977109e+02   1.59222004e+00  -6.39588294e-01   6.20233353e+01
  -2.04090831e+01   4.79611999e-03   7.19628745e-02  -9.23534904e-02
   1.74416751e-01   1.05035871e-01   4.58784064e-02  -7.61318481e+00
  -5.20400635e+00   1.48283596e+01   2.79244722e-01   6.82089298e-02
   8.56580415e+00]


### e. Fit a PCR model on the training set, with M chosen by cross-validation. Report the test error obtained, along with the value of M selected by cross-validation

In [320]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
pca = PCA()
linreg = LinearRegression()
pipe = Pipeline(steps=[('pca',pca),('linear',linreg)])

In [348]:
gscv = GridSearchCV(pipe, dict(pca__n_components=np.arange(1,college_trainX.shape[1]+1)),cv=5)
gscv.fit(college_trainX,college_trainY)
print gscv.score(college_testX,college_testY)
print gscv.best_params_

0.946463693802
{'pca__n_components': 17}


### f. Fit a PLS model on the training set, with M chosen by crossvalidation. Report the test error obtained, along with the value of M selected by cross-validation.

In [363]:
from sklearn.cross_decomposition import PLSRegression

In [371]:
pls = PLSRegression(n_components=1)
pls.fit(college_trainX,college_trainY)
pls.score(college_testX,college_testY)
# gscv = GridSearchCV(pls, dict(n_components=np.arange(1,college_trainX.shape[1]+1)),cv=5)
# gscv.fit(college_trainX,college_trainY)
# gscv.score(college_testX,college_testY)
# gscv.cv_results_

0.22380325138461199